In [1]:
# 필요한 라이브러리 설치
!pip install datasets torch torchvision scikit-learn matplotlib
!git clone https://github.com/HRNet/HRNet-Image-Classification
%cd HRNet-Image-Classification
!pip install -r requirements.txt
%cd ..

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from datasets import load_dataset
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt

# HRNet 모델 임포트 (리포지토리 구조에 따라 경로 조정 필요)
import sys
sys.path.append('HRNet-Image-Classification/lib')
from models.cls_hrnet import get_cls_net

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 데이터셋 로드
dataset = load_dataset("dwb2023/brain-tumor-image-dataset-semantic-segmentation")
train = dataset["train"]
test = dataset["test"]
valid = dataset["valid"]

# 이미지 전처리 파이프라인
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.convert('RGB') if x.mode != 'RGB' else x),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 커스텀 데이터셋 클래스
class BrainTumorDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        label = self.dataset[idx]["category_id"]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)
# 데이터로더 생성
train_dataset = BrainTumorDataset(train, transform=transform)
valid_dataset = BrainTumorDataset(valid, transform=transform)
test_dataset = BrainTumorDataset(test, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# HRNet 모델 초기화 및 수정
hrnet_config = {
    'MODEL': {
        'EXTRA': {
            'STAGE1': {
                'NUM_MODULES': 1,
                'NUM_BRANCHES': 1,
                'BLOCK': 'BOTTLENECK',
                'NUM_BLOCKS': [4],
                'NUM_CHANNELS': [64],
                'FUSE_METHOD': 'SUM'
            },
            'STAGE2': {
                'NUM_MODULES': 1,
                'NUM_BRANCHES': 2,
                'BLOCK': 'BASIC',
                'NUM_BLOCKS': [4,4],
                'NUM_CHANNELS': [48, 96],
                'FUSE_METHOD': 'SUM'
            },
            'STAGE3': {
                'NUM_MODULES': 4,
                'NUM_BRANCHES': 3,
                'BLOCK': 'BASIC',
                'NUM_BLOCKS': [4,4,4],
                'NUM_CHANNELS': [48, 96, 192],
                'FUSE_METHOD': 'SUM'
            },
            'STAGE4': {
                'NUM_MODULES': 3,
                'NUM_BRANCHES': 4,
                'BLOCK': 'BASIC',
                'NUM_BLOCKS': [4,4,4,4],
                'NUM_CHANNELS': [48, 96, 192, 384],
                'FUSE_METHOD': 'SUM'
            }
        },
        'PRETRAINED': '',
        'NUM_CLASSES': 1000
    }
}

# 수정된 모델 초기화
model = get_cls_net(hrnet_config)
model.classifier = nn.Linear(model.classifier.in_features, 1)

In [4]:
# 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 10
best_val_auc = 0

for epoch in range(num_epochs):
    # 학습 단계
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_loader.dataset)

    # 검증 단계
    model.eval()
    val_probs, val_labels = [], []
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            val_probs.extend(torch.sigmoid(outputs).cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    # 평가 지표 계산
    val_preds = (np.array(val_probs) > 0.5).astype(int)
    val_f1 = f1_score(val_labels, val_preds)
    tn, fp, fn, tp = confusion_matrix(val_labels, val_preds).ravel()
    val_iou = tp / (tp + fp + fn)
    val_auc = roc_auc_score(val_labels, val_probs)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val F1: {val_f1:.4f} | Val IoU: {val_iou:.4f} | Val AUC: {val_auc:.4f}")

    # 최고 모델 저장
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_model.pth')

# 테스트 평가
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
test_probs, test_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        test_probs.extend(torch.sigmoid(outputs).cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_preds = (np.array(test_probs) > 0.5).astype(int)
test_f1 = f1_score(test_labels, test_preds)
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
test_iou = tp / (tp + fp + fn)
test_auc = roc_auc_score(test_labels, test_probs)

print("\nFinal Test Performance:")
print(f"F1-score: {test_f1:.4f}")
print(f"IoU: {test_iou:.4f}")
print(f"ROC-AUC: {test_auc:.4f}")


"""
model : HRNet-W18-C
num_epochs = 10
Final Test Performance:
F1-score: 0.7087
IoU: 0.0000
ROC-AUC: 0.9218
"""

Epoch 1/10
Train Loss: -17.7786 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.5616
Epoch 2/10
Train Loss: -71.3731 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.7576
Epoch 3/10
Train Loss: -140.6225 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.9300
Epoch 4/10
Train Loss: -234.3184 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.9400
Epoch 5/10
Train Loss: -351.4954 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.4817
Epoch 6/10
Train Loss: -477.4906 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.9263
Epoch 7/10
Train Loss: -641.5166 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.8571
Epoch 8/10
Train Loss: -817.3460 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.8151
Epoch 9/10
Train Loss: -1036.3442 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.5594
Epoch 10/10
Train Loss: -1255.4032 | Val F1: 0.6573 | Val IoU: 0.0000 | Val AUC: 0.8201

Final Test Performance:
F1-score: 0.7087
IoU: 0.0000
ROC-AUC: 0.9218


ValueError: y_true takes value in {1.0, 2.0} and pos_label is not specified: either make y_true take value in {0, 1} or {-1, 1} or pass pos_label explicitly.